In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import warnings

%matplotlib inline

In [5]:
rest_data = pd.read_csv("C:/Users/aryan/OneDrive/Desktop/NYU DSGA3001/zomato.csv")

In [8]:
rest_data = rest_data.drop_duplicates('url')

In [9]:
rest_data.drop('menu_item',axis=1,inplace=True)

In [11]:
#21 null values in location column 
rest_data['location'].isna().sum()

#227 null in rest_type
rest_data['rest_type'].isna().sum()

#20252 null in dish_liked, 28078 null values in dish_liked column without removing any others
rest_data['dish_liked'].isna().sum()

#11 null in dish_liked, 45 null values in dish_liked column without removing any others
rest_data['cuisines'].isna().sum()

#11 null, 45 null values in column without removing any others
rest_data['cuisines'].isna().sum()

#247 null, 346 null values column without removing any others
rest_data['approx_cost(for two people)'].isna().sum()

#0 null, 0 null values column without removing any others
rest_data['reviews_list'].isna().sum()

#'menu_item' dropped in dataset

#0 null, 0 null values column without removing any others
rest_data['listed_in(type)'].isna().sum()

0

In [10]:
rest_data.dropna(subset=['rate','location','rest_type','cuisines','approx_cost(for two people)'],inplace=True)

In [ ]:
#Data cleaning: 
#Change pricing and any other numerical columns to integers
#Change rate to single number (not x/5)

In [12]:
#Remove commas from the numbers in the approx_cost
rest_data['approx_cost(for two people)'] = rest_data['approx_cost(for two people)'].str.replace(',', '')

In [13]:
rest_data['approx_cost(for two people)'] = rest_data['approx_cost(for two people)'].astype('Int64')

In [14]:
rest_data['approx_cost(for two people)'].describe()

count       43533.0
mean     594.850252
std      459.354032
min            40.0
25%           300.0
50%           450.0
75%           700.0
max          6000.0
Name: approx_cost(for two people), dtype: Float64

In [15]:
#Keep only the rating and convert to float (remove the '/5' for each entry)
rest_data['rate'] = rest_data['rate'].astype('str')

#Split string by '/'
rest_data['rate'] = rest_data['rate'].str.split('/').str[0]

#Convert to float
rest_data['rate'] = pd.to_numeric(rest_data['rate'], errors = 'coerce')
rest_data['rate']

0        4.1
1        4.1
2        3.8
3        3.7
4        3.8
        ... 
51709    3.7
51711    2.5
51712    3.6
51715    4.3
51716    3.4
Name: rate, Length: 43533, dtype: float64

In [16]:
#Make extra column for normalized ratings

#change phone column to flag of landline (080) or personal number (+91) (landline is a big restaurant = hypothesis)

#split cuisine types and create more rows for those that have multiples (make separate dataframe)

#Nice to have: Nominatim API for address to lat/long